<a href="https://colab.research.google.com/github/NonKhuna/DataPro-Algorithm/blob/main/thai_Datapro_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab is a program that was implemented according to https://www.aaai.org/Papers/AAAI/2000/AAAI00-093.pdf. 

# **Import**

## **library**

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# # release version 
# ! pip install pythainlp

In [3]:
import numpy as  np
import pandas as pd
import json
import re

# from queue import Queue

# import math
# from scipy.stats import norm
# import ipywidgets as widgets
# # from nltk.tokenize import word_tokenize
# from pythainlp import word_tokenize

In [4]:
# import nltk
# nltk.download('punkt')

In [5]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory


In [6]:
!pip install datapro-learning==1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Preprocess Data**

In [7]:
!wget https://data.go.th/dataset/723f4c28-79de-4a29-bea9-efacbaec236d/resource/db646c54-2780-4b06-bc27-d4455ad1a6f4/download/untitled.xlsx

--2022-07-08 07:24:02--  https://data.go.th/dataset/723f4c28-79de-4a29-bea9-efacbaec236d/resource/db646c54-2780-4b06-bc27-d4455ad1a6f4/download/untitled.xlsx
Resolving data.go.th (data.go.th)... 164.115.75.59
Connecting to data.go.th (data.go.th)|164.115.75.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274852 (268K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘untitled.xlsx.3’

untitled.xlsx.3     100%[===================>] 268.41K   225KB/s    in 1.2s    

2022-07-08 07:24:06 (225 KB/s) - ‘untitled.xlsx.3’ saved [274852/274852]



In [8]:
df = pd.read_excel("untitled.xlsx")

In [9]:
df.head()

,รหัสจังหวัด,รหัสสายทาง,ชื่อสายทาง,ระยะทาง(กม.),หน่วยรับผิดชอบ,ความกว้าง (เลน),เลน,วันที่ปรับปรุง
0,22,จบ.3035,แยก ทล.317 (กม.ที่ 21+350) - น้ำตกอ่างเบง,10.37570,สทช.3 ขทช.จันทบุรี,3.5,2,10/3/2013
1,22,จบ.1003,แยก ทล.3 (กม.ที่ 304+700) - บ.หนองบัว,9.40012,สทช.3 ขทช.จันทบุรี,3.5,2,10/3/2013
2,22,จบ.1008,แยก ทล.3 (กม.ที่ 297+300) - บ.สนามไชย,13.98180,สทช.3 บทช.เขาคิชฌกูฎ (จบ.),3.5,2,10/3/2013
3,22,จบ.1011,แยก ทล.3 (กม.ที่ 292+500) - บ.ช้างข้าม,9.50708,สทช.3 บทช.เขาคิชฌกูฎ (จบ.),3.5,2,10/3/2013
4,22,จบ.1018,แยก ทล.3 (กม.ที่ 301+500) - บ.คลองลาว,8.81050,สทช.3 บทช.เขาคิชฌกูฎ (จบ.),3.5,2,10/3/2013


In [10]:
df = df.dropna()

In [11]:
data_sample = df["รหัสสายทาง"]

# Main source code

## Class Program

### Class: Type's tree

In [7]:
# download Tree of type, json format
!gdown 1LIGUjJRCdjV_xh3WzFPEqOgogQNokt16

Downloading...
From: https://drive.google.com/uc?id=1LIGUjJRCdjV_xh3WzFPEqOgogQNokt16
To: /content/tree-type.json
100% 1.89k/1.89k [00:00<00:00, 1.47MB/s]


In [8]:
!mkdir datapro
!mv /content/tree-type.json datapro

mkdir: cannot create directory ‘datapro’: File exists


In [106]:
import json
import re
from queue import Queue
from pathlib import Path

class TreeOfType:
  """
  Instantiate a type's tree.
  This tree will used to assignt type to any token.

  :param path: path to tree-type store in json format.
  :type path: str
  """

  def __init__(self):
    self.data = dict()

    default_tree_type = Path('datapro/tree-type.json')
    self.load_tree_type(default_tree_type)
  
  def load_tree_type(self, path):
    '''
      Load new type's tree.

      :param path: path where json file store.
      :type path: str

      :return: The json datastructure as dict
      :rtype: dict
    '''
    filename = Path(path)
    if not filename.exists():
      print("Not found.")
    else :
      with open(filename, 'r') as f :
        tree = json.load(f)
      self.data = tree
      self.generate_code()
  
  def generate_code(self):
    '''
      Generate code for each node. consist of key and data. lower equal A, B, C ..
    '''
    _tree_type = self.data

    q = Queue()
    _tree_type["TOKEN"]['code'] = dict()
    _tree_type["TOKEN"]["code"]["key"] = '@'
    _tree_type["TOKEN"]["code"]["data"] = ''
    q.put("TOKEN")
    
    while not q.empty() :
      now_node = q.get()
      for i, child in enumerate(_tree_type[now_node]['children']) :
        now_key = ord(_tree_type[now_node]["code"]["key"])
        before_data = _tree_type[now_node]["code"]["data"]
        _tree_type[child]["code"] = dict()
        _tree_type[child]["code"]["key"] = chr(now_key+1)
        _tree_type[child]["code"]["data"] = before_data + "." + chr(now_key+1) + str(i+1)
        q.put(child)
    self.data = _tree_type
      
  def fix_format(self, token):
    """
      Fix start and end with only 1 token.

      :param token: token
      :type token: str

      :return: The token after fixed.
      :rtype: str
    """
    if len(token)>2 and token[0]=='^' and token[-1]=='$': return re.escape(token)
    return "^"+re.escape(token)+"$"

  def unfix_format(self, token) :
    """
      Unfix start and end. deleting "^" adn "$".

      :param token: token
      :type token: str

      :return: The token after fixed.
      :rtype: str
    """
    if len(token)>2 and (token[0]!='^' or token[-1]!='$'): return token
    return token[1:-1]

  def revese_escape(self, token) :
    """
      :param token: token
      :type token: str

      :return: The token after fixed.
      :rtype: str
    """
    return re.sub(r'\\(.)', r'\1', token)

  def follow_by(self, token, tokenT):
    """
      Check whather The tokenT is a generalization of the token.

      :param token: Target token.
      :type token: str

      :param tokenT: Target Type of token
      :type tokenT: str

      :return: The result of checking.
      :rtype: bool
    """
    return re.search(self.data[tokenT]["regex"], token)

  def sub_generalize(self, token1,token2): # O(1)
    """
      check whether token2 is a generalization of token1

      :param token1: Token1.
      :type token1: str

      :param token2: Token2
      :type token2: str

      :return: The result of checking.
      :rtype: bool
    """
    dif_level = abs(ord(self.data[token1]["code"]["key"]) - ord(self.data[token2]["code"]["key"]))
    return self.data[token2]["code"]["data"] in self.data[token1]["code"]["data"] and dif_level==1

  def create_tree_type_node(self, token_name, parent, regex):
    """
      Create new node to the type's tree.

      :param token_name: Name of new node.
      :type token_name: str

      :param parent: The parent node.
      :type parent: str

      :param regex: Regular expression of this node.
      :type regex: str
    """
    self.data[token_name] = dict()
    self.data[token_name]["regex"] = regex
    self.data[token_name]["children"] = []
    self.data[token_name]["parent"] = parent
    self.data[token_name]["code"] = dict()
    self.data[token_name]["code"]["key"] = chr(ord(self.data[parent]["code"]["key"]) + 1)
    self.data[token_name]["code"]["data"] = self.data[parent]["code"]["data"] + "." + self.data[token_name]["code"]["key"] + str(len(self.data[parent]["children"]))
    self.data[parent]["children"].append(token_name)

  def assign_type(self, token, grows_tree=False) :
    """
      Assign types to given token.

      :param token: The Target token.
      :type token: str

      :param grows_tree: The state that checkห whether new node can be produced. 
      :type grows_tree: bool

      :return: set of token's types.
      :rtype: set
    """
    # time complexity is O(height of type's tree)
    types = set()
    parent = "TOKEN" # last node that matching.

    _tree_type = self.data

    q = Queue()
    for k in _tree_type["TOKEN"]["children"] :
      q.put(k)

    while not q.empty() :
      now_type = q.get()
      if (now_type not in  _tree_type) : continue
      if re.search( _tree_type[now_type]["regex"], self.revese_escape(token)) :
        types.add(now_type)
        parent = now_type
        for k in _tree_type[now_type]["children"] :
          q.put(k)
    
    if parent=="DIGIT" : # new general type
      len_token = str(len(token))
      new_type = len_token + "_DIGIT"
      if not new_type in _tree_type :
        self.create_tree_type_node(new_type, parent, "^\\d{" + len_token + "}$")
        types.add(new_type)

    if  _tree_type[parent]["regex"] != self.fix_format(parent) and (token not in _tree_type) and grows_tree: # general node
        # constuct new special node
        self.create_tree_type_node(token, parent, self.fix_format(token))
        types.add(token)
    return types

  def print_space(self, k) :
    for i in range(len(k)) :
      if k[i][1]:
        print(" ", end="")
      else :
        print("|", end="")
      for j in range(k[i][0]):
        print(" ", end="")

  def display(self, root="TOKEN", level=0, stack_space=[]):
    """
      show typs's tree structure display.
    """
    if level == 0:
      print("---------- Type's Tree -------------")
      print("TOKEN")
    cnt = 0
    size = len(self.data[root]['children'])
    for child in self.data[root]['children'] :
      if child in self.data :
        self.print_space(stack_space)
        if cnt == size-1 :
          print("└── " + child)
        else :
          print("├── " + child)
        temp = stack_space.copy()
        temp.append([2 + int(len(child)/2), cnt == size-1])
        self.display(child, level+1, temp)
        cnt+=1
    

In [30]:
t = TreeOfType()
t.display()

datapro/tree-type.json
---------- Type's Tree -------------
TOKEN
├── PUNCT
└── ALPHANUM
       ├── ALPHA
       |    ├── CAPS
       |    ├── LOWER
       |    └── ALLCAPS
       └── NUMBER
             ├── DECIMAL
             └── INT
                 └── DIGIT


In [18]:
# test
ex = t.tree_type['DIGIT']['regex']
text = '1568'
x = re.findall(ex, text)
print(ex)

^\d+$


### Class: Node prefix tree

In [61]:
class Node:
  """
  Instantiate a prefix node.
  This will produce prefix tree to find sequence of pattern.

  :param token: The token of this node
  :type token: str

  :param examples: The observations that match with this pattern.
  :type examples: list

  :param pattern: A list of sequcent pattern.
  :type pattern: list

  :param parent_regex: Accumulates the regex of up to this node.
  :type parent_regex: str

  :param next_po: The next position or index that this node will consider.
  :type next_po: str

  :param parent: The parent node.
  :type parent: str
  """

  def __init__(self, token="", examples=[], pattern=[], parent_regex="", next_po="",parent=""):
    self.token = token
    self.examples = examples
    self.count = len(examples)
    self.pattern = pattern
    self.children = []
    self.regex = parent_regex
    self.next_po = next_po
    self.parent = parent

  def info(self):
    print("token:", self.token)
    print("amount of example:", self.count)
    print("pattern:", self.pattern)
    print("next_po:", self.next_po)

  def get_children_token_type(self, tree_type, dict_token, k):
    uniq_token = set()
    for ex in self.examples :
      # bound range
      if len(ex) > self.next_po :
          uniq_token.add(ex[self.next_po])
    
    set_token_type = set()
    for token in uniq_token :
      is_uniq = (dict_token[token] >= k)
      # print(token, dict_token[token], k, is_uniq)
      set_token_type = set_token_type.union(tree_type.assign_type(token, is_uniq))
    return set_token_type
  
  def print_space(self, k) :
    for i in range(len(k)) :
      if k[i][1]:
        print(" ", end="")
      else :
        print("|", end="")
      for j in range(k[i][0]):
        print(" ", end="")

  def display(self, level=0, stack_space=[]):
    cnt = 0
    size = len(self.children)
    for child in self.children :
      self.print_space(stack_space)
      if cnt == size-1 :
        print("└── " + child.token)
      else :
        print("├── " + child.token)
      temp = stack_space.copy()
      temp.append([2 + int(len(child.token)/2), cnt == size-1])
      child.display(level+1, temp)
      cnt+=1
    
  def find(self,name_token) :
    for child in self.children :
      if child.token == name_token : return child
      res = child.find(name_token) 
      if res : return res
    return None

### Class Datapro

In [65]:
# from .prefixNode import Node
# from .treeType import TreeOfType

from queue import Queue
from pythainlp import word_tokenize
from scipy.stats import norm
import math
import re

import pandas as pd

class DataPro:
  """
  Instantiate a Data pro model.
  The model will be trained by given data field, alpha and k_percentage.

  :param alpha: The alpha is used to test hypotheses.
  :type alpha: float

  :param k_percentage: The percentage of occuring data proportion with all token.
  :type k_percentage: int
  """
  def __init__(self, alpha=0.05, k_percentage=5):
    self.alpha = alpha
    self.k_percentage = k_percentage
    self.root = Node()
    self.tree_type = TreeOfType()
    self.res = []
    self.data = []
    self.tokenize_data = []
  
  def get_dic_token(self):
    # data is a list of list
    # O(nm) n = amount of data, m = lenght of each data.
    dictT = {}
    cnt = 0
    for row in self.tokenize_data :
      for token in row :
        tokenTs = self.tree_type.assign_type(token, False)
        tokenTs.add(token)
        for tokenT in tokenTs :
          if tokenT not in dictT :
            dictT[tokenT] = 1
          else :
            dictT[tokenT] +=1
        cnt+=1
    return dictT,cnt
  
  def tokenize(self, data):
    s = []
    for x in data :
      s.append(word_tokenize(str(x)))
    return s

  def significant(self, count1, count2, num_token, token_count, alpha):
    """
      param:
        - count1 : count seq that token C appear after seq Q
        - count2 : count seq Q
        - num_token : amount of token C
        - token_count : amount of all token in data field
        - alpha 
    """
    p1 = count1 / count2
    p2 = num_token / token_count # overall prob of occurrence token T 
    p_a = (count1 + num_token) / (count2 + token_count)
    try :
      z = (p1 -p2) / math.sqrt(p_a * (1-p_a) * (1/count2 + 1/token_count))
    except :
      z = 1e9 # case prop = 1
    cdf = norm.cdf(-z, 0, 1) # z-score
    return cdf < alpha

  def load_tree_type(self, path) :
    self.tree_type.load_tree_type(path)
  
  def fit(self, data) :
    self.data = data
    self.tokenize_data = self.tokenize(data)
    k = int((self.k_percentage/100) * len(self.data)) 

    q = Queue()
    self.root = Node("root", self.tokenize_data, [], "", 0, None)
    q.put(self.root)


    # Knowing how many token occuring in the data before assigned type.
    # ** token_count required before all seq token's type are generated. Actualy token
    dict_token, token_count = self.get_dic_token()

    while not q.empty():
        # node Q
        Q = q.get()

        # Create children Of Q 
        # Get all posible token that are children of Q
        all_token_type = Q.get_children_token_type(self.tree_type, dict_token, k)
        for tokenT in all_token_type :
          C = Node(tokenT, [], Q.pattern.copy(), Q.regex ,Q.next_po+1, Q)
          C.examples = [ex for ex in Q.examples if (Q.next_po < len(ex) and self.tree_type.follow_by(ex[Q.next_po],tokenT))]
          C.count = len(C.examples)
          C.pattern.append(tokenT)
          C.regex += self.tree_type.unfix_format(self.tree_type.data[tokenT]["regex"])
          if self.significant(C.count, Q.count, dict_token[tokenT], token_count, self.alpha) :
            Q.children.append(C)
        
        # Prune gerneralizations ?
        temp_children = []
        for C in Q.children :
          N = C.count
          for S in Q.children :
            if S.token != C.token and self.tree_type.sub_generalize(S.token, C.token):
              N -= S.count
          if self.significant(N, Q.count, dict_token[C.token], token_count, self.alpha) :
            temp_children.append(C)
            q.put(C)
        Q.children = temp_children

        # Determinize children of Q
        for C in Q.children :
          for S in Q.children :
            if S.token != C.token and self.tree_type.sub_generalize(S.token, C.token) :
              new_examples = []
              for x in C.examples :
                if x not in S.examples :
                  new_examples.append(x)
              C.examples = new_examples
          C.count = len(C.examples)

    # Extract patterns from tree
    self.res = []
    q = Queue()
    q.put(self.root)
    while not q.empty() :
      Q = q.get()
      for C in Q.children :
        N = C.count
        q.put(C)
        for S in C.children :
            N-=S.count
        if self.significant(N, Q.count, dict_token[C.token], token_count, self.alpha) :
          self.res.append(C.pattern)

  """
  Method for seeing the result.
  """

  def display_tree_type(self):
    self.tree_type.display()

  def display(self):
    self.root.display()
  
  def result(self):
    return self.res
  
  def evaluate_score(self) :
    def measure_matching(pattern) :
      '''
      This function compares the pattern with seq_token_data which is generated after tokenize.
      '''
      cnt = 0;
      size_pattern = len(pattern)
      for data in self.tokenize_data :
        if size_pattern == len(data) :
          is_match = True
          for token_i in range(size_pattern) :
            if not re.search(self.tree_type.data[pattern[token_i]]["regex"], data[token_i]):
              is_match = False
              break
          if is_match: cnt+=1
      return cnt/len(self.tokenize_data)
    res_df = pd.DataFrame(columns=["pattern", "score"])
    res_df["pattern"] = self.res
    res_df["score"] = res_df["pattern"].apply(lambda x: measure_matching(x))
    res_df = res_df.sort_values(by=["score"], ascending=False).reset_index()
    res_df.drop("index", axis=1, inplace=True)
    return res_df

# Example

In [12]:
from datapro import DataPro
# example Data
# data_sample = data_sample.astype(str)
data_sample

0       จบ.3035
1       จบ.1003
2       จบ.1008
3       จบ.1011
4       จบ.1018
         ...   
3259    ขก.2079
3263    มค.4017
3264    จบ.3019
3272    รอ.4002
3281    รย.5071
Name: รหัสสายทาง, Length: 2036, dtype: object

In [13]:
# Create Object DataPro
data_pro = DataPro(alpha=0.01, k_percentage=1)

# train with data
data_pro.fit(data_sample)

In [14]:
# show sorted result by matching pattern with data
s = data_pro.evaluate_score()
s

,pattern,score
0,"[ลบ, ., 4_DIGIT]",0.045187
1,"[อบ, ., 4_DIGIT]",0.040275
2,"[ศก., 4_DIGIT]",0.039784
3,"[ก, จ., 4_DIGIT]",0.038802
4,"[นม, ., 4_DIGIT]",0.035855
...,...,...
77,"[ลย., 4004]",0.000491
78,"[สห, ., 4004]",0.000491
79,"[สห, ., 4011]",0.000491
80,"[ป, จ., 4019]",0.000491


In [15]:
# show result
data_pro.result()

[['ลย.', '4_DIGIT'],
 ['ลย.', '4004'],
 ['รย.', '4_DIGIT'],
 ['สค.', '4010'],
 ['สค.', '4_DIGIT'],
 ['ยส.', '4010'],
 ['ยส.', '4_DIGIT'],
 ['ยส.', '4011'],
 ['ยส.', '4019'],
 ['ยส.', '4004'],
 ['กพ.', '4_DIGIT'],
 ['ตร.', '4010'],
 ['ตร.', '4_DIGIT'],
 ['ตร.', '4019'],
 ['พช.', '4_DIGIT'],
 ['อย.', '4_DIGIT'],
 ['ปข.', '4_DIGIT'],
 ['ชพ.', '4_DIGIT'],
 ['ฉช.', '4008'],
 ['ฉช.', '4_DIGIT'],
 ['ฉช.', '4011'],
 ['ชม.', '4_DIGIT'],
 ['ปท.', '4_DIGIT'],
 ['อต.', '4_DIGIT'],
 ['อต.', '4019'],
 ['ศก.', '4_DIGIT'],
 ['รบ.', '4_DIGIT'],
 ['รอ.', '4_DIGIT'],
 ['อจ.', '4_DIGIT'],
 ['อจ.', '4011'],
 ['อจ.', '4019'],
 ['อจ.', '4004'],
 ['อท.', '4010'],
 ['อท.', '4008'],
 ['อท.', '4_DIGIT'],
 ['อท.', '4004'],
 ['มค.', '4_DIGIT'],
 ['สส.', '4_DIGIT'],
 ['สส.', '4004'],
 ['สท.', '4_DIGIT'],
 ['พร', '.', '4_DIGIT'],
 ['สก', '.', '4_DIGIT'],
 ['กบ', '.', '4_DIGIT'],
 ['อบ', '.', '4_DIGIT'],
 ['ป', 'จ.', '4_DIGIT'],
 ['ป', 'จ.', '4019'],
 ['ก', 'จ.', '4_DIGIT'],
 ['ก', 'จ.', '4004'],
 ['สห', '.', '4_DIGI

In [103]:
s["pattern"][0]

['10', '/', '3', '/', '2013']

In [19]:
# display prefix tree structure
data_pro.display()

├── ลย.
|   ├── 4_DIGIT
|   └── 4004
├── พร
|   └── .
|      └── 4_DIGIT
├── สก
|   └── .
|      └── 4_DIGIT
├── กบ
|   └── .
|      └── 4_DIGIT
├── รย.
|   └── 4_DIGIT
├── อบ
|   └── .
|      └── 4_DIGIT
├── ป
|  └── จ.
|      ├── 4_DIGIT
|      └── 4019
├── ก
|  └── จ.
|      ├── 4_DIGIT
|      └── 4004
├── สค.
|   ├── 4010
|   └── 4_DIGIT
├── ยส.
|   ├── 4010
|   ├── 4_DIGIT
|   ├── 4011
|   ├── 4019
|   └── 4004
├── กพ.
|   └── 4_DIGIT
├── สห
|   └── .
|      ├── 4_DIGIT
|      ├── 4011
|      └── 4004
├── ตร.
|   ├── 4010
|   ├── 4_DIGIT
|   └── 4019
├── พ
|  └── จ.
|      └── 4_DIGIT
├── พช.
|   └── 4_DIGIT
├── อย.
|   └── 4_DIGIT
├── ช
|  └── บ.
|      └── 4_DIGIT
├── ปข.
|   └── 4_DIGIT
├── ตก
|   └── .
|      ├── 4008
|      ├── 4018
|      ├── 4_DIGIT
|      └── 4011
├── พบ
|   └── .
|      ├── 4008
|      ├── 4_DIGIT
|      ├── 4011
|      └── 4019
├── สบ
|   └── .
|      └── 4_DIGIT
├── นม
|   └── .
|      └── 4_DIGIT
├── พล
|   └── .
|      ├── 4018
|      └── 4_DIGIT
├── 

In [16]:
data_pro.load_tree_type("/content/tree-type - Copy.json")

In [18]:
data_pro.display_tree_type()

---------- Type's Tree -------------
TOKEN
├── PUNCT
└── ALPHANUM
       ├── ALPHA
       |    ├── CAPS
       |    ├── LOWER
       |    └── ALLCAPS
       └── NUMBER
             ├── DECIMAL
             └── INT
                 └── DIGIT
